In [1]:
import time

from sklearn.datasets import load_iris, fetch_20newsgroups, fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

In [2]:
# K近邻
"""
K-近邻预测用户签到位置
:return:None
"""
# 读取数据
data = pd.read_csv("train.csv")

print(data.head(10))
print(data.shape)
print(data.info())
# 处理数据
# 1、缩小数据,查询数据,为了减少计算时间
data = data.query("x > 2.0 &  x < 2.25 & y > 2.5 & y < 2.75")

   row_id       x       y  accuracy    time    place_id
0       0  0.7941  9.0809        54  470702  8523065625
1       1  5.9567  4.7968        13  186555  1757726713
2       2  8.3078  7.0407        74  322648  1137537235
3       3  7.3665  2.5165        65  704587  6567393236
4       4  4.0961  1.1307        31  472130  7440663949
5       5  3.8099  1.9586        75  178065  6289802927
6       6  6.3336  4.3720        13  666829  9931249544
7       7  5.7409  6.7697        85  369002  5662813655
8       8  4.3114  6.9410         3  166384  8471780938
9       9  6.3414  0.0758        65  400060  1253803156
(29118021, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29118021 entries, 0 to 29118020
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   row_id    int64  
 1   x         float64
 2   y         float64
 3   accuracy  int64  
 4   time      int64  
 5   place_id  int64  
dtypes: float64(2), int64(4)
memory usage: 1.3 GB
None


In [3]:
data.shape

(16068, 6)

In [4]:
data.describe()

,row_id,x,y,accuracy,time,place_id
count,1.606800e+04,16068.000000,16068.000000,16068.000000,16068.000000,1.606800e+04
mean,1.459072e+07,2.126385,2.626873,82.986557,420393.863829,5.608534e+09
std,8.420166e+06,0.073003,0.070087,118.557430,228646.985800,2.484970e+09
min,2.101000e+03,2.000100,2.500100,1.000000,58.000000,1.000617e+09
25%,7.295210e+06,2.062600,2.565675,24.000000,212949.750000,3.143663e+09
50%,1.463582e+07,2.124100,2.634700,62.000000,428691.500000,6.015971e+09
75%,2.192291e+07,2.192400,2.682300,76.000000,622458.000000,7.651100e+09
max,2.911724e+07,2.249900,2.749900,1010.000000,786211.000000,9.998058e+09


In [5]:
time_value = pd.to_datetime(data['time'], unit='s')
print(time_value.head(10))


2101    1970-01-04 23:46:01
2273    1970-01-05 14:06:01
3154    1970-01-05 00:30:19
4209    1970-01-06 21:00:22
8453    1970-01-01 23:16:23
9362    1970-01-08 04:57:48
10075   1970-01-08 08:31:59
10095   1970-01-02 10:09:20
10479   1970-01-08 23:27:34
12178   1970-01-08 06:29:41
Name: time, dtype: datetime64[ns]


In [6]:
time_value = pd.DatetimeIndex(time_value)

print('-' * 50)
print(time_value[0:10])

--------------------------------------------------
DatetimeIndex(['1970-01-04 23:46:01', '1970-01-05 14:06:01',
               '1970-01-05 00:30:19', '1970-01-06 21:00:22',
               '1970-01-01 23:16:23', '1970-01-08 04:57:48',
               '1970-01-08 08:31:59', '1970-01-02 10:09:20',
               '1970-01-08 23:27:34', '1970-01-08 06:29:41'],
              dtype='datetime64[ns]', name='time', freq=None)


In [7]:
print('-' * 50)
# 构造一些特征，执行的警告是因为我们的操作是复制，loc是直接放入
print(type(data))

--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>


In [8]:
data.insert(data.shape[1], 'day', time_value.day)
data.insert(data.shape[1], 'hour', time_value.hour)#是否去一个地方打卡，早上，中午，晚上是有影响的
data.insert(data.shape[1], 'weekday', time_value.weekday) 

In [9]:
data = data.drop(['time'], axis=1)
print('-' * 50)
data.head()

--------------------------------------------------


,row_id,x,y,accuracy,place_id,day,hour,weekday
2101,2101,2.2326,2.6215,55,2119626770,4,23,6
2273,2273,2.0903,2.6981,166,8054130031,5,14,0
3154,3154,2.2383,2.7305,64,1727040747,5,0,0
4209,4209,2.2111,2.5478,22,5689129232,6,21,1
8453,8453,2.2012,2.6419,42,7926938948,1,23,3


In [10]:
per = pd.Period('2025-01-10 18:00', 'h')
per.weekday

4

In [11]:
data.describe()

,row_id,x,y,accuracy,place_id,day,hour,weekday
count,1.606800e+04,16068.000000,16068.000000,16068.000000,1.606800e+04,16068.000000,16068.000000,16068.000000
mean,1.459072e+07,2.126385,2.626873,82.986557,5.608534e+09,5.367065,11.462659,3.133869
std,8.420166e+06,0.073003,0.070087,118.557430,2.484970e+09,2.644881,6.906369,1.765577
min,2.101000e+03,2.000100,2.500100,1.000000,1.000617e+09,1.000000,0.000000,0.000000
25%,7.295210e+06,2.062600,2.565675,24.000000,3.143663e+09,3.000000,6.000000,2.000000
50%,1.463582e+07,2.124100,2.634700,62.000000,6.015971e+09,5.000000,12.000000,3.000000
75%,2.192291e+07,2.192400,2.682300,76.000000,7.651100e+09,8.000000,17.000000,4.000000
max,2.911724e+07,2.249900,2.749900,1010.000000,9.998058e+09,10.000000,23.000000,6.000000


In [12]:
place_count = data.groupby('place_id').aggregate(np.count_nonzero)
place_count

,row_id,x,y,accuracy,day,hour,weekday
place_id,,,,,,,
1000616752,2,2,2,2,2,2,2
1017703558,10,10,10,10,10,10,5
1068339748,1,1,1,1,1,1,1
1068896566,1,1,1,1,1,1,1
1096341648,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...
9977423098,1,1,1,1,1,1,1
9980711012,1,1,1,1,1,1,1
9986238568,1,1,1,1,1,1,1


In [13]:
place_count['x'].describe()

count    912.000000
mean      17.618421
std       66.373037
min        1.000000
25%        1.000000
50%        2.000000
75%        5.000000
max      824.000000
Name: x, dtype: float64

In [14]:
tf = place_count[place_count.row_id > 3].reset_index()
tf  #剩余的签到地点

,place_id,row_id,x,y,accuracy,day,hour,weekday
0,1017703558,10,10,10,10,10,10,5
1,1097200869,16,16,16,16,16,16,14
2,1099012307,120,120,120,120,120,117,97
3,1104074781,4,4,4,4,4,4,4
4,1188903087,8,8,8,8,8,8,7
...,...,...,...,...,...,...,...,...
265,9904182060,18,18,18,18,18,16,14
266,9919777984,11,11,11,11,11,11,9
267,9940471456,68,68,68,68,68,62,58
268,9950190890,6,6,6,6,6,6,6


In [15]:
data = data[data['place_id'].isin(tf.place_id)]
data.shape

(15148, 8)

In [16]:
# # 取出数据当中的特征值和目标值
y = data['place_id']
# 删除目标值，保留特征值，
x = data.drop(['place_id'], axis=1)
# 删除无用的特征值，row_id是索引,这就是噪音
x = x.drop(['row_id'], axis=1)
print(x.shape)
print(x.columns)

(15148, 6)
Index(['x', 'y', 'accuracy', 'day', 'hour', 'weekday'], dtype='object')


In [17]:
# 进行数据的分割训练集合测试集
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1)

# 特征工程（标准化）,下面3行注释，一开始我们不进行标准化，看下效果，目标值要不要标准化？
std = StandardScaler()
# #
# # # 对测试集和训练集的特征值进行标准化,服务于knn fit
x_train = std.fit_transform(x_train)
# # transform返回的是copy，不在原有的输入对象中去修改
# print(id(x_test))
print(std.mean_)
print(std.var_)
x_test = std.transform(x_test)  #transfrom不再进行均值和方差的计算，是在原有的基础上去标准化
print('-' * 50)
# print(id(x_test))
print(std.mean_)
print(std.var_)

[ 2.12678283  2.62717618 82.58128686  5.36255611 11.4637796   3.13211865]
[5.37196722e-03 4.84314746e-03 1.36584127e+04 6.97928275e+00
 4.80807451e+01 3.12742627e+00]
--------------------------------------------------
[ 2.12678283  2.62717618 82.58128686  5.36255611 11.4637796   3.13211865]
[5.37196722e-03 4.84314746e-03 1.36584127e+04 6.97928275e+00
 4.80807451e+01 3.12742627e+00]


In [18]:
x_train.shape

(11361, 6)

In [19]:
# # 进行算法流程 # 超参数，可以通过设置n_neighbors=5，来调整结果好坏
knn = KNeighborsClassifier(n_neighbors=3)

# # fit， predict,score，训练，knn的fit是不训练的，只是把训练集的特征值和目标值放入到内存中
knn.fit(x_train, y_train)
# # #
# # # 得出预测结果
y_predict = knn.predict(x_test)
# #
print("预测的目标签到位置为：", y_predict[0:10])
# # #
# # # # 得出准确率,是评估指标
print("预测的准确率:", knn.score(x_test, y_test))
# print(y_predict)
# y_test

预测的目标签到位置为： [7175626357 1888235595 5728378850 7346312886 1974067055 7076275776
 2719042180 6015970893 7709179486 3926374830]
预测的准确率: 0.38737787166622656


In [20]:
print(y_test[0:10])

22061214    7175626357
24397369    1285051622
8590496     9350693064
28639329    5126068724
22889622    2719042180
27852840    7766976999
10029824    2719042180
21909733    6015970893
26850072    6502963776
8940516     3926374830
Name: place_id, dtype: int64


In [22]:
#网格搜索时讲解
# # 构造一些参数（超参）的值进行搜索
param = {"n_neighbors": [3, 5, 7, 9, 11, 13, 16],'weights':['uniform', 'distance']}
#
# 进行网格搜索，cv=3是3折交叉验证，用其中2折训练，1折验证
gc = GridSearchCV(knn, param_grid=param, cv=3)

gc.fit(x_train, y_train)  #你给它的x_train，它又分为训练集，验证集

# 预测准确率，为了给大家看看
print("在测试集上准确率：", gc.score(x_test, y_test))

print("在交叉验证当中最好的结果：", gc.best_score_) #最好的结果

print("选择最好的模型是：", gc.best_estimator_) #最好的模型,告诉你用了哪些参数

print("每个超参数每次交叉验证的结果：")
gc.cv_results_

C:\Program Files\Python312\Lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


在测试集上准确率： 0.4248745709004489
在交叉验证当中最好的结果： 0.4011970777220315
选择最好的模型是： KNeighborsClassifier(n_neighbors=11, weights='distance')
每个超参数每次交叉验证的结果：


{'mean_fit_time': array([0.00733733, 0.00534884, 0.00632644, 0.00691644, 0.00738645,
        0.00621033, 0.00550723, 0.0058403 , 0.00550318, 0.00601204,
        0.0061686 , 0.00600449, 0.00537968, 0.00701078]),
 'std_fit_time': array([0.00191298, 0.00026514, 0.00025795, 0.00123844, 0.00104601,
        0.00023226, 0.00079889, 0.00047205, 0.00040863, 0.00010714,
        0.00087859, 0.00081577, 0.00027357, 0.00216874]),
 'mean_score_time': array([0.09380873, 0.04405363, 0.10698708, 0.05135059, 0.11233505,
        0.05894327, 0.10069601, 0.06647182, 0.11536908, 0.07603463,
        0.10971697, 0.07262381, 0.10897811, 0.08952959]),
 'std_score_time': array([6.08917350e-03, 7.05860053e-04, 1.81761314e-02, 2.86620974e-03,
        6.89855138e-03, 1.12862860e-03, 9.18833433e-05, 4.63884326e-03,
        7.64137090e-03, 3.20627069e-03, 2.96715689e-03, 6.19546641e-04,
        1.17581163e-03, 1.33197514e-03]),
 'param_n_neighbors': masked_array(data=[3, 3, 5, 5, 7, 7, 9, 9, 11, 11, 13, 13, 16, 16],
